# Particle Tracking with PRT

In [ ]:
from pathlib import Path

import flopy as fp
import numpy as np
import xarray as xr

import nlmod

In [ ]:
nlmod.util.get_color_logger("INFO")
nlmod.show_versions()

In [ ]:
model_ws = Path("./scratch_model")
model_name = "from_scratch"

# create new subdir for PRT model
prt_ws = Path(model_ws) / "prt"

In [ ]:
ds = xr.open_dataset(model_ws / f"{model_name}.nc")

In [ ]:
# Create a new Simulation object
simprt = nlmod.sim.sim(ds, sim_ws=prt_ws)
tdis = nlmod.sim.tdis(ds, simprt)

# Add PRT model
prt = nlmod.prt.prt(ds, simprt)

# DIS: discretization package
dis = nlmod.prt.dis(ds, prt)

# MIP: matrix input package(?)
mip = nlmod.prt.mip(ds, prt, porosity=0.3)

# PRP: particle release package
pd = fp.modpath.ParticleData(
    [
        (k, i, j)
        for i in np.arange(0, ds.sizes["y"], 11)
        for j in np.arange(0, ds.sizes["x"], 11)
        for k in [0, 1, 2]
    ],
    structured=True,
)
releasepts = list(pd.to_prp(prt.modelgrid, global_xy=False))
prp = nlmod.prt.prp(ds, prt, packagedata=releasepts, perioddata={0: ["FIRST"]})

# FMI: flow model interface
gwf_budget_file = Path(ds.model_ws) / f"{model_name}.cbc"
gwf_head_file = Path(ds.model_ws) / f"{model_name}.hds"
fmi = fp.mf6.ModflowPrtfmi(
    prt,
    packagedata=[
        ("GWFHEAD", gwf_head_file.absolute().resolve()),
        ("GWFBUDGET", gwf_budget_file.absolute().resolve()),
    ],
)

# OC: output control
oc = nlmod.prt.oc(ds, prt)

# EMS: explicit model solution
ems = nlmod.sim.ems(simprt, model=prt)

In [ ]:
simprt.write_simulation()
simprt.run_simulation()

Load pathline data

In [ ]:
from flopy.utils.prtfile import PathlineFile

trkfile = PathlineFile(
    Path(prt.model_ws) / f"{prt.name}.trk",
    header_filename=Path(prt.model_ws) / f"{prt.name}.trk.hdr",
)
trk = trkfile.get_alldata()
trk_df = trkfile.get_dataframe()

Plot results

In [ ]:
pmv = fp.plot.PlotMapView(prt)
# loop to avoid different tracks get connected in plot
for i in range(len(trk)):
    pmv.plot_pathline(trk[i:i+1], layer="all", colors="k", lw=1.0)
pmv.plot_endpoint(trk_df, color="r", marker=".", direction="starting", zorder=10);